In [ ]:
import pandas as pd
import polars as pl
import requests

In [ ]:
# dr.conn.sql("DROP TABLE DP03Table;")

In [ ]:
_year = 2023
tmp = dr.pull_query(
                params=[
                    "DP03_0001E",
                    "DP03_0008E",
                    "DP03_0009E",
                    "DP03_0014E",
                    "DP03_0016E",
                    "DP03_0019E",
                    "DP03_0025E",
                    "DP03_0051E",
                    "DP03_0052E",
                    "DP03_0053E",
                    "DP03_0054E",
                    "DP03_0055E",
                    "DP03_0056E",
                    "DP03_0057E",
                    "DP03_0058E",
                    "DP03_0059E",
                    "DP03_0060E",
                    "DP03_0061E",
                    "DP03_0070E",
                    "DP03_0074E",
                ],
                year=_year,
            )
tmp = tmp.rename(
    {
        "dp03_0001e": "total_population",
        "dp03_0008e": "in_labor_force",
        "dp03_0009e": "unemployment",
        "dp03_0014e": "own_children6",
        "dp03_0016e": "own_children17",
        "dp03_0019e": "commute_car",
        "dp03_0025e": "commute_time",
        "dp03_0051e": "total_house",
        "dp03_0052e": "inc_less_10k",
        "dp03_0053e": "inc_10k_15k",
        "dp03_0054e": "inc_15k_25k",
        "dp03_0055e": "inc_25k_35k",
        "dp03_0056e": "inc_35k_50k",
        "dp03_0057e": "inc_50k_75k",
        "dp03_0058e": "inc_75k_100k",
        "dp03_0059e": "inc_100k_150k",
        "dp03_0060e": "inc_150k_200k",
        "dp03_0061e": "inc_more_200k",
        "dp03_0070e": "with_social_security",
        "dp03_0074e": "food_stamp",
    }
)
tmp = tmp.rename({"zip code tabulation area": "zipcode"}).drop(
    ["state"]
)

In [ ]:
def pull_query(params: list, year: int) -> pl.DataFrame:
    # prepare custom census query
    param = ",".join(params)
    base = "https://api.census.gov/data/"
    flow = "/acs/acs5/profile"
    url = f"{base}{year}{flow}?get={param}&for=zip%20code%20tabulation%20area:*"
    df = pl.DataFrame(requests.get(url).json())

    # get names from DataFrame
    names = df.select(pl.col("column_0")).transpose()
    names = names.to_dicts().pop()
    names = dict((k, v.lower()) for k, v in names.items())

    # Pivot table
    df = df.drop("column_0").transpose()
    return df.rename(names).with_columns(year=pl.lit(year))

def pull_dp03() -> pl.DataFrame:
    if "DP03Table" not in self.conn.sql("SHOW TABLES;").df().get("name").tolist():
        init_dp03_table(self.data_file)
    for _year in range(2011, 2023):
        if (
            self.conn.sql(f"SELECT * FROM 'DP03Table' WHERE year={_year}")
            .df()
            .empty
        ):
            logging.info(f"pulling {_year} data")
            tmp = self.pull_query(
                params=[
                    "DP03_0001E",
                    "DP03_0008E",
                    "DP03_0009E",
                    "DP03_0014E",
                    "DP03_0016E",
                    "DP03_0019E",
                    "DP03_0025E",
                    "DP03_0051E",
                    "DP03_0052E",
                    "DP03_0053E",
                    "DP03_0054E",
                    "DP03_0055E",
                    "DP03_0056E",
                    "DP03_0057E",
                    "DP03_0058E",
                    "DP03_0059E",
                    "DP03_0060E",
                    "DP03_0061E",
                    "DP03_0070E",
                    "DP03_0074E",
                ],
                year=_year,
            )
            tmp = tmp.rename(
                {
                    "dp03_0001e": "total_population",
                    "dp03_0008e": "in_labor_force",
                    "dp03_0009e": "unemployment",
                    "dp03_0014e": "own_children6",
                    "dp03_0016e": "own_children17",
                    "dp03_0019e": "commute_car",
                    "dp03_0025e": "commute_time",
                    "dp03_0051e": "total_house",
                    "dp03_0052e": "inc_less_10k",
                    "dp03_0053e": "inc_10k_15k",
                    "dp03_0054e": "inc_15k_25k",
                    "dp03_0055e": "inc_25k_35k",
                    "dp03_0056e": "inc_35k_50k",
                    "dp03_0057e": "inc_50k_75k",
                    "dp03_0058e": "inc_75k_100k",
                    "dp03_0059e": "inc_100k_150k",
                    "dp03_0060e": "inc_150k_200k",
                    "dp03_0061e": "inc_more_200k",
                    "dp03_0070e": "with_social_security",
                    "dp03_0074e": "food_stamp",
                }
            )
            tmp = tmp.rename({"zip code tabulation area": "zipcode"}).drop(
                ["state"]
            )
            logging.info(f"succesfully inserting {_year}")
            # except:
            #     logging.warning(f"The ACS for {_year} is not availabe")
            #     continue
        else:
            logging.info(f"data for {_year} is in the database")
            continue

In [ ]:
pull_query(["*"], 2022)